<a href="https://colab.research.google.com/github/prajakta311/git---session/blob/main/Sentiment_Analysis_(NLP_and_LSTM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import spacy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [2]:
data = pd.read_csv('Sentiment.csv')

In [3]:
data.sample(5)

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
3221,3222,No candidate mentioned,0.4235,yes,0.6508,Neutral,0.6508,None of the above,0.4235,NaN,...,NaN,2,NaN,NaN,RT @NiktaKanuka: Check out @klipfolio's #GOPDe...,NaN,07-08-2015 09:21,6.300000e+17,"Ottawa, Canada",Atlantic Time (Canada)
8485,8486,Jeb Bush,1.0000,yes,1.0000,Negative,1.0000,FOX News or Moderators,1.0000,NaN,...,NaN,153,NaN,NaN,RT @RWSurferGirl: Fox is cherry picking the ca...,NaN,07-08-2015 01:14,6.300000e+17,NaN,NaN
8472,8473,Chris Christie,1.0000,yes,1.0000,Negative,1.0000,None of the above,0.6630,NaN,...,NaN,48,NaN,NaN,RT @RWSurferGirl: Tonight we learned that @Gov...,NaN,07-08-2015 01:40,6.300000e+17,"Shenandoah, TX",NaN
12254,12255,No candidate mentioned,0.6382,yes,1.0000,Neutral,0.6382,None of the above,0.6382,NaN,...,NaN,19,NaN,NaN,RT @BethBehrs: Classy. #GOPDebates https://t....,NaN,06-08-2015 19:24,6.290000e+17,The Sunny Philippines,Alaska
8579,8580,No candidate mentioned,1.0000,yes,1.0000,Neutral,0.6304,FOX News or Moderators,1.0000,NaN,...,NaN,116,NaN,NaN,RT @RWSurferGirl: So @megynkelly posed for adu...,NaN,06-08-2015 23:34,6.300000e+17,California's Central Coast,Pacific Time (US & Canada)


In [4]:
data = data.loc[:, [ 'candidate', 'sentiment', 'text' ]]

In [5]:
data.head(5)

,candidate,sentiment,text
0,No candidate mentioned,Neutral,RT @NancyLeeGrahn: How did everyone feel about...
1,Scott Walker,Positive,RT @ScottWalker: Didn't catch the full #GOPdeb...
2,No candidate mentioned,Neutral,RT @TJMShow: No mention of Tamir Rice and the ...
3,No candidate mentioned,Positive,RT @RobGeorge: That Carly Fiorina is trending ...
4,Donald Trump,Positive,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...


In [6]:
#Model which will take the tweet data and classify it into sentiment
data.text

0        RT @NancyLeeGrahn: How did everyone feel about...
1        RT @ScottWalker: Didn't catch the full #GOPdeb...
2        RT @TJMShow: No mention of Tamir Rice and the ...
3        RT @RobGeorge: That Carly Fiorina is trending ...
4        RT @DanScavino: #GOPDebate w/ @realDonaldTrump...
                               ...                        
13866    RT @cappy_yarbrough: Love to see men who will ...
13867    RT @georgehenryw: Who thought Huckabee exceede...
13868    RT @Lrihendry: #TedCruz As President, I will a...
13869    RT @JRehling: #GOPDebate Donald Trump says tha...
13870    RT @Lrihendry: #TedCruz headed into the Presid...
Name: text, Length: 13871, dtype: object

In [7]:
nlp = spacy.load("en_core_web_sm")

In [8]:
def remove_stop_word(tweet):
  tweet = nlp(tweet)
  tweet = " ".join([word.text for word in tweet if not word.is_stop])
  return tweet


In [9]:
data.text.apply(func=remove_stop_word)

0        RT @NancyLeeGrahn : feel Climate Change questi...
1        RT @ScottWalker : catch # GOPdebate night . Sc...
2        RT @TJMShow : mention Tamir Rice # GOPDebate h...
3        RT @RobGeorge : Carly Fiorina trending -- hour...
4        RT @DanScavino : # GOPDebate w/ @realDonaldTru...
                               ...                        
13866    RT @cappy_yarbrough : Love men faced pregnancy...
13867    RT @georgehenryw : thought Huckabee exceeded e...
13868    RT @Lrihendry : # TedCruz President , tell tru...
13869    RT @JRehling : # GOPDebate Donald Trump says t...
13870    RT @Lrihendry : # TedCruz headed Presidential ...
Name: text, Length: 13871, dtype: object

In [10]:
def pre_processing(tweet):
  word_list = []
  tweet = nlp(tweet)
  for word in tweet:
    if word.is_stop == False:
      if word.is_punct == False:
        word_list.append(word.text)
  text = " ".join(word_list)
  text = text.replace("RT @", " ")
  text = text.lower()
  return text

In [11]:
print(data.text[10])
pre_processing(data.text[10])

@JGreenDC @realDonaldTrump In all fairness #BillClinton owns that phrase.#GOPDebate


'@jgreendc @realdonaldtrump fairness billclinton owns phrase.#gopdebate'

In [12]:
data.groupby(["candidate", "sentiment"]).size()

candidate               sentiment
Ben Carson              Negative      186
                        Neutral        54
                        Positive      164
Chris Christie          Negative      218
                        Neutral        42
                        Positive       33
Donald Trump            Negative     1758
                        Neutral       446
                        Positive      609
Jeb Bush                Negative      589
                        Neutral        72
                        Positive       44
John Kasich             Negative       82
                        Neutral        47
                        Positive      113
Marco Rubio             Negative      105
                        Neutral        51
                        Positive      119
Mike Huckabee           Negative      237
                        Neutral        83
                        Positive       73
No candidate mentioned  Negative     4724
                        Neutral      2087


In [13]:
data["text_process"] = data.text.apply(func=pre_processing)

In [ ]:
#Just like images tweet(text) can have different sizes
#Solution in images was to resize the images to common size, similarly in text we resize them

In [14]:
import tensorflow as tf

In [15]:
# Set max_feature
max_feature = 2000  # or any other value you want

# Tokenizer Configuration
token = tf.keras.preprocessing.text.Tokenizer(num_words=max_feature)
token.fit_on_texts(data.text_process)

# Vocabulary Size Check
vocab_size = len(token.word_index)
print("Vocabulary size:", vocab_size)

# Adjust max_feature if needed
max_feature = min(2000, vocab_size)

# Input Sequences
X = token.texts_to_sequences(data.text_process)
X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=31)  # Add maxlen parameter to pad_sequences

# Check max index in X
max_index_X = np.max(X)
print("Max index in X:", max_index_X)

# Model Configuration
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=max_feature, output_dim=128, input_length=31))
model.add(tf.keras.layers.SpatialDropout1D(0.5))
model.add(tf.keras.layers.LSTM(256, dropout=0.8))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

model.summary()

# Model Compilation and Training
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Y = pd.get_dummies(data.sentiment)
model.fit(X, Y, epochs=10, batch_size=64, validation_split=0.2)

Vocabulary size: 18896
Max index in X: 1999
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 31, 128)           256000    
                                                                 
 spatial_dropout1d (Spatial  (None, 31, 128)           0         
 Dropout1D)                                                      
                                                                 
 lstm (LSTM)                 (None, 256)               394240    
                                                                 
 dense (Dense)               (None, 3)                 771       
                                                                 
Total params: 651011 (2.48 MB)
Trainable params: 651011 (2.48 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
174/174 [======================